# NFA Implementation
This is a simple NFA implementation. It reflects the definition of an NFA as discussed in class. It is `not` the final implementation our implementation of `grep` will use.

NFA representation:
- Number of states `num_states` (the state set is then 0..num_states -1)
- Start state `start_state`
- Boolean array `is_accepting` storing for every state whether it's accepting
- Transition function `delta` represented as a dictionary with `(state, character)` keys and `state` sets as values
- a key of `(state, None)` in the transition function represents an $\epsilon$-transition out of `state`

In [12]:
# %load src/nfa_simple.py

class NFA:
    def __init__(self, num_states, start_state, is_accepting, delta):
        self.num_states = num_states
        self.start_state = start_state
        self.delta = delta
        self.is_accepting = is_accepting

    def accepts(self, string):
        states = self.eclose({self.start_state})

        for ch in string:
            states = self.transition(states, ch)

        return any(self.is_accepting[state] for state in states)

    def eclose(self, states):
        closure = states
        closure_size = 0

        # if there are more epsilon transitions after union
        while len(closure) > closure_size:
            closure_size = len(closure)
            closure = closure.union(*(self.__successors(state, None) for state in closure))

        return closure

    def transition(self, states, ch):
        successors = set().union(*(self.__successors(state, ch) for state in states))
        return self.eclose(successors)

    def __successors(self, state, ch):
        # if there is no value, return empty set
        return self.delta.get((state, ch), set())

    

# An Example NFA
The NFA decides the language of all decimal strings that contain the substring `3136`

In [13]:
nfa = NFA (
    5,
    0,
    [False, False, False, False, True],
    {
        # Transtitions out of s0
        (0, '0'): {0}, (0, '1'): {0},
        (0, '2'): {0}, (0, '3'): {0, 1},
        (0, '4'): {0}, (0, '5'): {0},
        (0, '6'): {0}, (0, '7'): {0},
        (0, '8'): {0}, (0, '9'): {0},

        # Transtitions out of s1
        (1, '1'): {2},

        # Transtitions out of s2
        (2, '3'): {3},

        # Transitions out of s3
        (3, '6'): {4},

        # Transitions out of s4
        (4, '0'): {4}, (4, '1'): {4},
        (4, '2'): {4}, (4, '3'): {4},
        (4, '4'): {4}, (4, '5'): {4},
        (4, '6'): {4}, (4, '7'): {4},
        (4, '8'): {4}, (4, '9'): {4},
    }
)

# Two Test Runs
The string `38119849183846313631148791193847` has `3136` as a substring, so it should be accepted

In [14]:
nfa.accepts("38119849183846313631148791193847")

True

The string `38119849183846311131148791193847` does not have `3136` as a substring, so it should be rejected

In [15]:
nfa.accepts("38119849183846311131148791193847")

False